# 🏘️ Main Street Negotiation - Comprehensive Analysis Dashboard

Complete analysis of Main Street real estate negotiations (Fred Starr vs Rosalind Cain)

**Analysis Sections:**
1. 📋 All Negotiation Records Overview
2. 👥 Student Demographics & Characteristics Analysis
3. 💰 Configuration Comparison: Base vs M vs M+P (Price Analysis)
4. 🧠 Memory (M) Evolution Tracking
5. 🎯 Planning (P) Strategy Tracking
6. 💬 Full Transcript Viewer
7. 📊 Performance Metrics Dashboard

---

## Setup

In [ ]:
import sqlite3
import pandas as pd
import json
import numpy as np
from datetime import datetime
from IPython.display import display, HTML, Markdown
import warnings
warnings.filterwarnings('ignore')

# ==================== CONFIGURATION ====================
DB_PATH = "negotiations.db"  # ⚠️ CHANGE THIS TO YOUR DATABASE PATH
SCENARIO_FILTER = "Main_Street"  # Focus on Main Street scenario

conn = sqlite3.connect(DB_PATH)
print(f"✅ Connected to: {DB_PATH}")
print(f"🎯 Analyzing scenario: {SCENARIO_FILTER}")

In [ ]:
# Helper Functions
def safe_json_load(json_str):
    if not json_str or json_str == 'null':
        return None
    try:
        return json.loads(json_str)
    except:
        return None

def calculate_duration(start, end):
    try:
        return (pd.to_datetime(end) - pd.to_datetime(start)).total_seconds() / 60
    except:
        return None

def get_config_label(use_memory, use_plan):
    if use_memory and use_plan:
        return 'M+P'
    elif use_memory:
        return 'M'
    elif use_plan:
        return 'P'
    else:
        return 'Base'

def print_header(title, char='=', width=100):
    print("\n" + char * width)
    print(f"{title.center(width)}")
    print(char * width + "\n")

def print_subheader(title, char='-', width=80):
    print("\n" + char * width)
    print(title)
    print(char * width)

print("✅ Helper functions loaded")

---
## 1. 📋 All Negotiation Records Overview

Complete list of all Main Street negotiations

In [ ]:
# Load all Main Street sessions
query_all = f"""
SELECT 
    session_id,
    student_role,
    ai_model,
    student_goes_first,
    use_memory,
    use_plan,
    total_rounds,
    deal_reached,
    deal_failed,
    status,
    created_at,
    updated_at,
    major,
    gender,
    negotiation_experience
FROM negotiation_sessions
WHERE scenario_name = '{SCENARIO_FILTER}'
ORDER BY created_at DESC
"""

df_all = pd.read_sql_query(query_all, conn)

# Add derived columns
df_all['Config'] = df_all.apply(lambda r: get_config_label(r['use_memory'], r['use_plan']), axis=1)
df_all['Duration (min)'] = df_all.apply(lambda r: calculate_duration(r['created_at'], r['updated_at']), axis=1)
df_all['Outcome'] = df_all.apply(lambda r: 
    '✅ Deal' if r['deal_reached'] 
    else '❌ Failed' if r['deal_failed'] 
    else '⏸️ Incomplete', axis=1)

df_all['Role'] = df_all['student_role'].map({
    'side1': '🏢 Buyer (Fred)',
    'side2': '🏘️ Seller (Rosalind)'
})

print_header("📋 ALL MAIN STREET NEGOTIATIONS")
print(f"Total Sessions: {len(df_all)}")
print(f"Successful Deals: {df_all['deal_reached'].sum()}")
print(f"Failed Negotiations: {df_all['deal_failed'].sum()}")
print(f"Incomplete: {(~df_all['deal_reached'] & ~df_all['deal_failed']).sum()}")

print(f"\n📊 Configuration Breakdown:")
print(df_all['Config'].value_counts().to_string())

print(f"\n🎭 Role Distribution:")
print(df_all['Role'].value_counts().to_string())

# Display table
display_df = df_all[[
    'session_id', 'Role', 'Config', 'Outcome', 
    'Duration (min)', 'created_at'
]].copy()
display_df['session_id'] = display_df['session_id'].str[:8]
display_df['created_at'] = pd.to_datetime(display_df['created_at']).dt.strftime('%Y-%m-%d %H:%M')

print("\n" + "="*100)
display(display_df)

---
## 2. 👥 Student Demographics & Characteristics Analysis

In [ ]:
print_header("👥 STUDENT DEMOGRAPHICS & CHARACTERISTICS")

# Filter out rows with no demographic data
df_demo = df_all.copy()

# Major Analysis
print_subheader("📚 Academic Major Distribution")
if df_demo['major'].notna().sum() > 0:
    major_counts = df_demo['major'].value_counts()
    print(f"Total with major data: {df_demo['major'].notna().sum()}")
    print("\nMajor breakdown:")
    for major, count in major_counts.items():
        pct = count / len(df_demo) * 100
        print(f"  {major}: {count} ({pct:.1f}%)")
else:
    print("No major data available")

# Gender Analysis
print_subheader("⚧ Gender Distribution")
if df_demo['gender'].notna().sum() > 0:
    gender_counts = df_demo['gender'].value_counts()
    print(f"Total with gender data: {df_demo['gender'].notna().sum()}")
    print("\nGender breakdown:")
    for gender, count in gender_counts.items():
        pct = count / len(df_demo) * 100
        print(f"  {gender}: {count} ({pct:.1f}%)")
else:
    print("No gender data available")

# Experience Analysis
print_subheader("💼 Negotiation Experience Distribution")
if df_demo['negotiation_experience'].notna().sum() > 0:
    exp_counts = df_demo['negotiation_experience'].value_counts()
    print(f"Total with experience data: {df_demo['negotiation_experience'].notna().sum()}")
    print("\nExperience breakdown:")
    for exp, count in exp_counts.items():
        pct = count / len(df_demo) * 100
        print(f"  {exp}: {count} ({pct:.1f}%)")
else:
    print("No experience data available")

# Success rate by demographics
print_subheader("📈 Success Rates by Demographics")

if df_demo['major'].notna().sum() > 0:
    print("\nBy Major:")
    major_success = df_demo.groupby('major')['deal_reached'].agg(['sum', 'count', 'mean'])
    major_success.columns = ['Deals', 'Total', 'Success Rate']
    major_success['Success Rate'] = (major_success['Success Rate'] * 100).round(1)
    print(major_success.to_string())

if df_demo['gender'].notna().sum() > 0:
    print("\nBy Gender:")
    gender_success = df_demo.groupby('gender')['deal_reached'].agg(['sum', 'count', 'mean'])
    gender_success.columns = ['Deals', 'Total', 'Success Rate']
    gender_success['Success Rate'] = (gender_success['Success Rate'] * 100).round(1)
    print(gender_success.to_string())

if df_demo['negotiation_experience'].notna().sum() > 0:
    print("\nBy Negotiation Experience:")
    exp_success = df_demo.groupby('negotiation_experience')['deal_reached'].agg(['sum', 'count', 'mean'])
    exp_success.columns = ['Deals', 'Total', 'Success Rate']
    exp_success['Success Rate'] = (exp_success['Success Rate'] * 100).round(1)
    print(exp_success.to_string())

---
## 3. 💰 Configuration Comparison: Base vs M vs M+P

Analysis of deal prices across different AI configurations

In [ ]:
# Load successful deals with price data
query_deals = f"""
SELECT 
    session_id,
    student_role,
    ai_role,
    use_memory,
    use_plan,
    student_deal_json,
    ai_deal_json,
    transcript,
    created_at,
    updated_at
FROM negotiation_sessions
WHERE scenario_name = '{SCENARIO_FILTER}' AND deal_reached = 1
ORDER BY created_at DESC
"""

df_deals = pd.read_sql_query(query_deals, conn)

# Parse deal JSONs and extract prices
def extract_price(deal_json_str):
    deal = safe_json_load(deal_json_str)
    if deal and 'final_price' in deal:
        return deal['final_price']
    return None

df_deals['student_price'] = df_deals['student_deal_json'].apply(extract_price)
df_deals['ai_price'] = df_deals['ai_deal_json'].apply(extract_price)
df_deals['config'] = df_deals.apply(lambda r: get_config_label(r['use_memory'], r['use_plan']), axis=1)
df_deals['duration'] = df_deals.apply(lambda r: calculate_duration(r['created_at'], r['updated_at']), axis=1)

# Count rounds
def count_rounds(transcript_json):
    transcript = safe_json_load(transcript_json)
    return len(transcript) // 2 if transcript else 0

df_deals['rounds'] = df_deals['transcript'].apply(count_rounds)

# Map roles
df_deals['student_role_label'] = df_deals['student_role'].map({
    'side1': 'Buyer (Fred)',
    'side2': 'Seller (Rosalind)'
})

df_deals['ai_role_label'] = df_deals['ai_role'].map({
    'side1': 'Buyer (Fred)',
    'side2': 'Seller (Rosalind)'
})

print_header("💰 DEAL PRICE ANALYSIS BY CONFIGURATION")

if len(df_deals) == 0:
    print("❌ No successful deals found")
else:
    print(f"Total Successful Deals: {len(df_deals)}\n")
    
    # Overall statistics
    print_subheader("📊 Overall Price Statistics")
    print(f"Average Final Price: ${df_deals['student_price'].mean():,.0f}")
    print(f"Median Final Price: ${df_deals['student_price'].median():,.0f}")
    print(f"Min Price: ${df_deals['student_price'].min():,.0f}")
    print(f"Max Price: ${df_deals['student_price'].max():,.0f}")
    print(f"Std Dev: ${df_deals['student_price'].std():,.0f}")
    
    # Price by configuration
    print_subheader("🔧 Average Price by Configuration")
    
    config_stats = df_deals.groupby('config').agg({
        'student_price': ['count', 'mean', 'median', 'std', 'min', 'max']
    }).round(0)
    
    print("\nConfiguration Summary:")
    print("="*80)
    for config in ['Base', 'M', 'P', 'M+P']:
        if config in config_stats.index:
            stats = config_stats.loc[config, 'student_price']
            print(f"\n{config}:")
            print(f"  Count: {int(stats['count'])} deals")
            print(f"  Average: ${stats['mean']:,.0f}")
            print(f"  Median:  ${stats['median']:,.0f}")
            print(f"  Range:   ${stats['min']:,.0f} - ${stats['max']:,.0f}")
            if stats['count'] > 1:
                print(f"  Std Dev: ${stats['std']:,.0f}")
    
    # Price by role and configuration
    print_subheader("🎭 Average Price by Role & Configuration")
    
    role_config_stats = df_deals.groupby(['student_role_label', 'config'])['student_price'].agg(['count', 'mean', 'median']).round(0)
    
    print("\nWhen Student is BUYER (Fred Starr):")
    print("  Goal: Minimize price (lower is better)")
    print("  BATNA: $675,000 (will walk away if price > $675k)")
    print("-"*80)
    if 'Buyer (Fred)' in role_config_stats.index.get_level_values(0):
        buyer_stats = role_config_stats.loc['Buyer (Fred)']
        for config in buyer_stats.index:
            stats = buyer_stats.loc[config]
            print(f"  {config}: ${stats['mean']:,.0f} avg (n={int(stats['count'])})")
    
    print("\nWhen Student is SELLER (Rosalind Cain):")
    print("  Goal: Maximize price (higher is better)")
    print("  BATNA: $475,000 (will walk away if price < $475k)")
    print("-"*80)
    if 'Seller (Rosalind)' in role_config_stats.index.get_level_values(0):
        seller_stats = role_config_stats.loc['Seller (Rosalind)']
        for config in seller_stats.index:
            stats = seller_stats.loc[config]
            print(f"  {config}: ${stats['mean']:,.0f} avg (n={int(stats['count'])})")
    
    # Detailed deal list
    print_subheader("📋 All Deals - Detailed List")
    
    deal_display = df_deals[[
        'session_id', 'student_role_label', 'config', 
        'student_price', 'rounds', 'duration'
    ]].copy()
    
    deal_display['session_id'] = deal_display['session_id'].str[:8]
    deal_display['student_price'] = deal_display['student_price'].apply(lambda x: f"${x:,.0f}")
    deal_display['duration'] = deal_display['duration'].round(1)
    
    deal_display.columns = ['Session', 'Student Role', 'Config', 'Final Price', 'Rounds', 'Duration (min)']
    
    display(deal_display)

### 3.1 Price Comparison Summary Table

In [ ]:
# Create comprehensive comparison table
if len(df_deals) > 0:
    print("\n" + "="*100)
    print("COMPREHENSIVE PRICE COMPARISON: BASE vs M vs M+P".center(100))
    print("="*100 + "\n")
    
    comparison_data = []
    
    for config in ['Base', 'M', 'P', 'M+P']:
        config_data = df_deals[df_deals['config'] == config]
        
        if len(config_data) > 0:
            comparison_data.append({
                'Configuration': config,
                'N Deals': len(config_data),
                'Avg Price': f"${config_data['student_price'].mean():,.0f}",
                'Median Price': f"${config_data['student_price'].median():,.0f}",
                'Min Price': f"${config_data['student_price'].min():,.0f}",
                'Max Price': f"${config_data['student_price'].max():,.0f}",
                'Avg Rounds': f"{config_data['rounds'].mean():.1f}",
                'Avg Duration': f"{config_data['duration'].mean():.1f} min"
            })
    
    if comparison_data:
        comparison_df = pd.DataFrame(comparison_data)
        display(comparison_df)
        
        print("\n💡 Key Insights:")
        print("-"*80)
        
        # Calculate buyer vs seller average for each config
        for config in ['Base', 'M', 'P', 'M+P']:
            config_data = df_deals[df_deals['config'] == config]
            if len(config_data) > 0:
                buyer_avg = config_data[config_data['student_role'] == 'side1']['student_price'].mean()
                seller_avg = config_data[config_data['student_role'] == 'side2']['student_price'].mean()
                
                print(f"\n{config}:")
                if not pd.isna(buyer_avg):
                    print(f"  When student is buyer: ${buyer_avg:,.0f} (student wants lower)")
                if not pd.isna(seller_avg):
                    print(f"  When student is seller: ${seller_avg:,.0f} (student wants higher)")
else:
    print("No deal data available for comparison")

---
## 4. 🧠 Memory (M) Evolution Tracking

Track how AI memory evolves round-by-round

In [ ]:
# Load sessions with memory
query_memory = f"""
SELECT 
    session_id,
    student_role,
    use_memory,
    use_plan,
    ai_memory,
    ai_memory_history,
    transcript,
    deal_reached
FROM negotiation_sessions
WHERE scenario_name = '{SCENARIO_FILTER}' 
  AND use_memory = 1 
  AND ai_memory_history IS NOT NULL
ORDER BY created_at DESC
"""

df_memory = pd.read_sql_query(query_memory, conn)

print_header("🧠 AI MEMORY EVOLUTION TRACKING")

if len(df_memory) == 0:
    print("❌ No sessions with memory tracking found")
else:
    print(f"Sessions with Memory: {len(df_memory)}\n")
    
    # Select a session to analyze (most recent by default)
    session_idx = 0  # ⚠️ Change this to analyze different sessions
    
    session = df_memory.iloc[session_idx]
    
    print(f"📌 Analyzing Session: {session['session_id'][:8]}...")
    print(f"   Student Role: {session['student_role']}")
    print(f"   Configuration: {get_config_label(session['use_memory'], session['use_plan'])}")
    print(f"   Deal Reached: {'✅ Yes' if session['deal_reached'] else '❌ No'}\n")
    
    memory_history = safe_json_load(session['ai_memory_history'])
    
    if memory_history:
        print(f"Total Memory Updates: {len(memory_history)}\n")
        print("="*100)
        
        for i, memory_state in enumerate(memory_history):
            round_info = memory_state.get('round', f'Update {i+1}')
            content = memory_state.get('content', '')
            
            print(f"\n🧠 MEMORY STATE #{i+1} (Round {round_info})")
            print("-"*100)
            print(content)
            print("-"*100)
        
        # Final memory
        print(f"\n🎯 FINAL MEMORY STATE")
        print("="*100)
        current_memory = safe_json_load(session['ai_memory'])
        if current_memory:
            print(current_memory)
        else:
            print(session['ai_memory'] or "(empty)")
        print("="*100)
        
        # Analysis
        transcript = safe_json_load(session['transcript'])
        if transcript:
            total_rounds = len(transcript) // 2
            print(f"\n📊 Memory Evolution Analysis:")
            print(f"   Total Rounds: {total_rounds}")
            print(f"   Memory Updates: {len(memory_history)}")
            print(f"   Updates per Round: {len(memory_history)/total_rounds:.2f}")
    else:
        print("❌ No memory history available")
    
    # List all sessions for reference
    print("\n\n" + "="*100)
    print("All Sessions with Memory (change session_idx above to analyze different sessions):")
    print("="*100)
    for idx, row in df_memory.iterrows():
        config = get_config_label(row['use_memory'], row['use_plan'])
        status = "✅ Deal" if row['deal_reached'] else "❌ No Deal"
        print(f"{idx}. {row['session_id'][:8]}... - {config} - {status}")

---
## 5. 🎯 Planning (P) Strategy Tracking

Track how AI planning evolves round-by-round

In [ ]:
# Load sessions with planning
query_plan = f"""
SELECT 
    session_id,
    student_role,
    use_memory,
    use_plan,
    ai_plan,
    ai_plan_history,
    transcript,
    deal_reached
FROM negotiation_sessions
WHERE scenario_name = '{SCENARIO_FILTER}' 
  AND use_plan = 1 
  AND ai_plan_history IS NOT NULL
ORDER BY created_at DESC
"""

df_plan = pd.read_sql_query(query_plan, conn)

print_header("🎯 AI PLANNING STRATEGY TRACKING")

if len(df_plan) == 0:
    print("❌ No sessions with planning tracking found")
else:
    print(f"Sessions with Planning: {len(df_plan)}\n")
    
    # Select a session to analyze
    session_idx = 0  # ⚠️ Change this to analyze different sessions
    
    session = df_plan.iloc[session_idx]
    
    print(f"📌 Analyzing Session: {session['session_id'][:8]}...")
    print(f"   Student Role: {session['student_role']}")
    print(f"   Configuration: {get_config_label(session['use_memory'], session['use_plan'])}")
    print(f"   Deal Reached: {'✅ Yes' if session['deal_reached'] else '❌ No'}\n")
    
    plan_history = safe_json_load(session['ai_plan_history'])
    
    if plan_history:
        print(f"Total Plan Updates: {len(plan_history)}\n")
        print("="*100)
        
        for i, plan_state in enumerate(plan_history):
            round_info = plan_state.get('round', f'Update {i+1}')
            content = plan_state.get('content', '')
            
            print(f"\n🎯 PLAN STATE #{i+1} (Round {round_info})")
            print("-"*100)
            print(content)
            print("-"*100)
        
        # Final plan
        print(f"\n🏁 FINAL PLAN STATE")
        print("="*100)
        current_plan = safe_json_load(session['ai_plan'])
        if current_plan:
            print(current_plan)
        else:
            print(session['ai_plan'] or "(empty)")
        print("="*100)
        
        # Analysis
        transcript = safe_json_load(session['transcript'])
        if transcript:
            total_rounds = len(transcript) // 2
            print(f"\n📊 Planning Evolution Analysis:")
            print(f"   Total Rounds: {total_rounds}")
            print(f"   Plan Updates: {len(plan_history)}")
            print(f"   Updates per Round: {len(plan_history)/total_rounds:.2f}")
    else:
        print("❌ No plan history available")
    
    # List all sessions
    print("\n\n" + "="*100)
    print("All Sessions with Planning (change session_idx above to analyze different sessions):")
    print("="*100)
    for idx, row in df_plan.iterrows():
        config = get_config_label(row['use_memory'], row['use_plan'])
        status = "✅ Deal" if row['deal_reached'] else "❌ No Deal"
        print(f"{idx}. {row['session_id'][:8]}... - {config} - {status}")

---
## 6. 💬 Full Transcript Viewer

Display complete conversation transcripts

In [ ]:
# Select a session to view transcript
session_to_view = 0  # ⚠️ Change this to view different sessions

if len(df_deals) > 0:
    session = df_deals.iloc[session_to_view]
    
    print_header("💬 FULL CONVERSATION TRANSCRIPT")
    
    print(f"Session: {session['session_id'][:8]}...")
    print(f"Student Role: {session['student_role_label']}")
    print(f"AI Role: {session['ai_role_label']}")
    print(f"Configuration: {session['config']}")
    print(f"Final Price: ${session['student_price']:,.0f}")
    print(f"Rounds: {session['rounds']}")
    print(f"Duration: {session['duration']:.1f} minutes\n")
    
    transcript = safe_json_load(session['transcript'])
    
    if transcript:
        print("="*100)
        
        for i, message in enumerate(transcript):
            # Parse message format: "Round X.Y - Label: content"
            if " - " in message:
                parts = message.split(" - ", 1)
                round_info = parts[0]
                
                if len(parts) > 1 and ": " in parts[1]:
                    label, content = parts[1].split(": ", 1)
                    
                    # Determine icon
                    if "Fred" in label or "Buyer" in label:
                        icon = "🏢"
                    elif "Rosalind" in label or "Seller" in label:
                        icon = "🏘️"
                    else:
                        icon = "💬"
                    
                    print(f"\n{icon} {round_info} - {label}")
                    print("-"*100)
                    print(content)
                else:
                    print(f"\n💬 {round_info}")
                    print("-"*100)
                    print(parts[1] if len(parts) > 1 else message)
            else:
                print(f"\n💬 Message {i+1}")
                print("-"*100)
                print(message)
        
        print("\n" + "="*100)
        print(f"\n✅ Deal reached at price: ${session['student_price']:,.0f}")
    else:
        print("❌ No transcript available")
    
    # List all successful deals for reference
    print("\n\n" + "="*100)
    print("All Successful Deals (change session_to_view above to view different transcripts):")
    print("="*100)
    for idx, row in df_deals.iterrows():
        price = row['student_price']
        config = row['config']
        role = row['student_role_label']
        print(f"{idx}. {row['session_id'][:8]}... - {role} - {config} - ${price:,.0f}")
else:
    print("❌ No successful deals to display")

---
## 7. 📊 Performance Metrics Dashboard

Comprehensive performance metrics across all sessions

In [ ]:
print_header("📊 PERFORMANCE METRICS DASHBOARD")

# Overall success rates
print_subheader("✅ Success Rates by Configuration")
success_by_config = df_all.groupby('Config').agg({
    'deal_reached': ['sum', 'count', 'mean']
})
success_by_config.columns = ['Successful', 'Total', 'Success Rate']
success_by_config['Success Rate'] = (success_by_config['Success Rate'] * 100).round(1)
print(success_by_config.to_string())

# Duration analysis
if len(df_deals) > 0:
    print_subheader("⏱️ Duration Analysis (Successful Deals Only)")
    duration_by_config = df_deals.groupby('config')['duration'].agg(['count', 'mean', 'median', 'min', 'max'])
    duration_by_config = duration_by_config.round(1)
    duration_by_config.columns = ['N', 'Mean (min)', 'Median (min)', 'Min (min)', 'Max (min)']
    print(duration_by_config.to_string())
    
    # Rounds analysis
    print_subheader("🔄 Rounds Analysis (Successful Deals Only)")
    rounds_by_config = df_deals.groupby('config')['rounds'].agg(['count', 'mean', 'median', 'min', 'max'])
    rounds_by_config = rounds_by_config.round(1)
    rounds_by_config.columns = ['N', 'Mean', 'Median', 'Min', 'Max']
    print(rounds_by_config.to_string())

# Efficiency metrics
print_subheader("⚡ Efficiency: Minutes per Round")
if len(df_deals) > 0:
    df_deals['min_per_round'] = df_deals['duration'] / df_deals['rounds']
    efficiency = df_deals.groupby('config')['min_per_round'].agg(['mean', 'median']).round(2)
    efficiency.columns = ['Mean (min/round)', 'Median (min/round)']
    print(efficiency.to_string())

# Value distribution by role
if len(df_deals) > 0:
    print_subheader("💰 Price Distribution by Student Role")
    
    print("\nWhen Student is BUYER (wants lower price):")
    buyer_deals = df_deals[df_deals['student_role'] == 'side1']
    if len(buyer_deals) > 0:
        buyer_stats = buyer_deals.groupby('config')['student_price'].agg(['count', 'mean', 'median'])
        buyer_stats['mean'] = buyer_stats['mean'].apply(lambda x: f"${x:,.0f}")
        buyer_stats['median'] = buyer_stats['median'].apply(lambda x: f"${x:,.0f}")
        buyer_stats.columns = ['N', 'Mean Price', 'Median Price']
        print(buyer_stats.to_string())
    else:
        print("  No data")
    
    print("\nWhen Student is SELLER (wants higher price):")
    seller_deals = df_deals[df_deals['student_role'] == 'side2']
    if len(seller_deals) > 0:
        seller_stats = seller_deals.groupby('config')['student_price'].agg(['count', 'mean', 'median'])
        seller_stats['mean'] = seller_stats['mean'].apply(lambda x: f"${x:,.0f}")
        seller_stats['median'] = seller_stats['median'].apply(lambda x: f"${x:,.0f}")
        seller_stats.columns = ['N', 'Mean Price', 'Median Price']
        print(seller_stats.to_string())
    else:
        print("  No data")

print("\n" + "="*100)

---
## 8. Close Database Connection

In [ ]:
conn.close()
print("✅ Database connection closed")